In [19]:
import folium
import shapely.wkt
import json
from geopy.distance import geodesic
import pandas as pd

In [11]:
df_pandas = pd.read_parquet('data')

# Display the first few rows
print(df_pandas.head())

                                        api_poi_id  \
0  17869_11329554616376164352_11224744174636137823   
1  17869_11329554655809175552_13152072077601261347   
2   17869_11329644549808128000_9320603266051224117   
3  17869_11329646544609673216_12959887055347796539   
4   17869_11329826997042216960_9790038048596750301   

                  reference_id    matching_run_id release_version provider_id  \
0  ChIJbQKLqmGXagwR8mjuzA0wqVA  20241126-00590024    24480.000 ON          om   
1  ChIJ7eeq7V6XagwRqIYhaLjyzzg  20241126-00590024    24480.000 ON          om   
2  ChIJR_QML0CLQAwRX2zdeWyK5b0  20241126-00590024    24480.000 ON          om   
3  ChIJj2TiLtWTQAwROBjUEG-y2j0  20241126-00590024    24480.000 ON          om   
4  ChIJV0pfe8uiQAwRnPXoc-7Z3a4  20241126-00590024    24480.000 ON          om   

     enduser_category category_id_tt                 category_name  \
0  Things to see & do           9902                Amusement Park   
1  Things to see & do           9902            

In [12]:
filtered_df = df_pandas[df_pandas["provider_routing_points"].apply(lambda x: len(x) > 1)]
filtered_df = filtered_df[filtered_df["reference_routing_points"].apply(lambda x: len(x) > 1)]

In [13]:
filtered_df.head()

,api_poi_id,reference_id,matching_run_id,release_version,provider_id,enduser_category,category_id_tt,category_name,reference_translated_cat_id,providers_translated_cat_id,...,region,reference_routing_points,country,provider_response,provider_routing_points,rpav_matching,rpav_matching_run_id,query_lat,query_lon,sample_run_id
24209,76397748250,ChIJESXsBKbGDQ0RKwPBeraQvz8,20240209-22044310,24060.000 OV,om,Health Care,7321,Hospital/Polyclinic,7321,7321,...,"{""country"": ""ESP"", ""a1_admin_code"": null, ""adm...","[[36.6994737, -6.146485999999999], [36.6986097...",ESP,"{""responses"" : [{""id"" : 83436245484, ""lat"" : 3...","[[36.698055, -6.1480527], [36.699245, -6.14893...",{'fields': {'distance_matrix': [[210.68105608 ...,20240216-15544023,None,None,None
24260,83576632843,ChIJIYFgZbYhQg0ROV8VlcrGG-c,20240209-22044310,24060.000 OV,om,Shopping,7373,Shopping Center,7373,7373,...,"{""country"": ""ESP"", ""a1_admin_code"": null, ""adm...","[[40.2709444, -3.6949843], [40.2711297, -3.697...",ESP,"{""responses"" : [{""id"" : 83381566541, ""lat"" : 4...","[[40.27122, -3.6974943], [40.273075, -3.6976647]]",{'fields': {'distance_matrix': [[215.16107076 ...,20240216-15544023,None,None,None
24362,49423943060,ChIJedXaFdwkQg0R2-BMyAo3-cs,20240209-22044310,24060.000 OV,om,Shopping,7373,Shopping Center,7373,9361,...,"{""country"": ""ESP"", ""a1_admin_code"": null, ""adm...","[[40.3680552, -3.6011447], [40.3708911, -3.597...",ESP,"{""responses"" : [{""id"" : 82009320719, ""lat"" : 4...","[[40.36901, -3.5990353], [40.36745, -3.5975916]]",{'fields': {'distance_matrix': [[207.91989441 ...,20240216-15544023,None,None,None
24377,83795330623,ChIJE9JYe6Y3Yg0RzWsSQZQSQsw,20240209-22044310,24060.000 OV,om,Health Care,7321,Hospital/Polyclinic,7321,9663,...,"{""country"": ""ESP"", ""a1_admin_code"": null, ""adm...","[[38.35291180000001, -0.4765705], [38.3530048,...",ESP,"{""responses"" : [{""id"" : 82585028023, ""lat"" : 3...","[[38.353027, -0.4763831], [38.35296, -0.4764822]]",{'fields': {'distance_matrix': [[20.78777025 ...,20240216-15544023,None,None,None
24383,84154595663,ChIJd_z05hGjpBIRl_5UAkf7vNo,20240209-22044310,24060.000 OV,om,Shopping,7373,Shopping Center,7373,7373,...,"{""country"": ""ESP"", ""a1_admin_code"": null, ""adm...","[[41.3896711, 2.198145], [41.39081119999999, 2...",ESP,"{""responses"" : [{""id"" : 83217614967, ""lat"" : 4...","[[41.39015, 2.1975386], [41.390106, 2.1987557]]",{'fields': {'distance_matrix': [[73.3000607 70...,20240216-15544023,None,None,None


In [14]:
for index, row in df_pandas.iterrows():
    if len(row['reference_routing_points']) > 1 & len(row['provider_routing_points']) > 2:
        print(f"Place: {index}, Routing Points: {len(row['reference_routing_points'])} and {len(row['provider_routing_points'])}")

In [15]:
#Features to add
# circle around poi loc if they rp and poi are the same
# polygon for poi geometry
# show google maps results for some examples

In [16]:
def provider_char_d(res):
    return res['responses'][0]['lat'], res['responses'][0]['lon']

def provider_latlon_(res):
    return res['responses'][0]['lat'], res['responses'][0]['lon']

def max_distance(centroid, markers):
    max_dist = 0
    # Compare every pair of markers
    for i in markers:
            dist = geodesic(centroid, i).km
            if dist > max_dist:
                max_dist = dist

    return max_dist*1000  # Return the distance in meters

# Function to calculate bounding box from a point and a distance (in meters)
def calculate_bounds(lat, lon, distance_meters):
    # Approximate the distance in degrees (1 degree of latitude is about 111 km)
    lat_offset = distance_meters / 111320  # 1 degree of latitude ~ 111320 meters
    lon_offset = distance_meters / (40008000 * (1 / 360)) * (1 / (111320 * 2))  # Roughly adjust for longitude

    # Calculate the bounding box
    min_lat = lat - lat_offset
    max_lat = lat + lat_offset
    min_lon = lon - lon_offset
    max_lon = lon + lon_offset

    return [[min_lat, min_lon], [max_lat, max_lon]]

In [17]:
def mean_distance_to_routing_points(poi_latlon, routing_points):
    distances = []
    for rp in routing_points:
        distance = geodesic(poi_latlon, rp).meters
        distances.append(distance)
    mean_distance = sum(distances) / len(distances) if distances else 0
    return mean_distance

In [18]:
row = df_pandas.iloc[24377]

rppa = row['rpav_matching']['fields']['rppa']
poi_characteristic_distance = row['rpav_matching']['fields']['poi_characteristic_distance']
assignation = row['rpav_matching']['fields']['assignation']
provider_routing_points = row["provider_routing_points"]
reference_latlon = (float(row['ref_lat']), float(row['ref_lon']))
provider_latlon = provider_latlon_(json.loads(row['provider_response']))  
reference_routing_points =  row["reference_routing_points"]

map_centroid = reference_latlon
width, height = 800, 600 
m = folium.Map(location=map_centroid, zoom_start=17, tiles='openstreetmap', width=width, height=height)

folium.Marker(location= reference_latlon, icon=folium.Icon(color='black', icon = "")).add_to(m)
folium.Marker(location= provider_latlon, icon=folium.Icon(color='red', icon = "")).add_to(m)

markers = [reference_latlon, provider_latlon]

for rp in provider_routing_points:
    folium.Circle(location=rp, radius=0.7*poi_characteristic_distance, color="red").add_to(m)
    folium.CircleMarker(location=rp, radius=4, color="red", fill=False, fill_color="red", fill_opacity=1).add_to(m)

    markers.append(rp)
    folium.PolyLine(locations=[rp, provider_latlon], color="red", weight=2, opacity=1, dashArray="5, 5").add_to(m)

if rppa > 0:
    for asign in assignation:
        if geodesic(reference_routing_points[asign[0]], provider_routing_points[asign[1]]).m < 0.7*poi_characteristic_distance:
            folium.PolyLine(locations=[reference_routing_points[asign[0]], provider_routing_points[asign[1]]], color="green", weight=4, opacity=1).add_to(m)

for rp in reference_routing_points:
    folium.CircleMarker(location=rp, radius=4, color="black", fill=True, fill_color="black", fill_opacity=1).add_to(m)
    markers.append(rp)
    folium.PolyLine(locations=[rp, reference_latlon], color="black", weight=2, opacity=1, dashArray="5, 5").add_to(m)


    
bounds = calculate_bounds(map_centroid[0], map_centroid[1], 1.5*max_distance(map_centroid, markers))
m.fit_bounds(bounds)

m